# Merging

- We need to merge in pandas if structure of DFs are not the same (equivalent of JOIN for SQL)

- E.g combinign registrations and logins (online vs physical site)
- We first need to decide on what column to merge together.

- inner merge: results need to be in both tables

In [1]:
import numpy as np
import pandas as pd

In [2]:
registrations = pd.DataFrame({'reg_id':[1,2,3,4],'name':['Andrew','Bobo','Claire','David']})
logins = pd.DataFrame({'log_id':[1,2,3,4],'name':['Xavier','Andrew','Yolanda','Bobo']})

In [3]:
logins

,log_id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


In [4]:
# A lot of docs on merge!
help(pd.merge)

Help on function merge in module pandas.core.reshape.merge:

merge(left, right, how: str = 'inner', on=None, left_on=None, right_on=None, left_index: bool = False, right_index: bool = False, sort: bool = False, suffixes=('_x', '_y'), copy: bool = True, indicator: bool = False, validate=None) -> 'DataFrame'
    Merge DataFrame or named Series objects with a database-style join.
    
    The join is done on columns or indexes. If joining columns on
    columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
    on indexes or indexes on a column or columns, the index will be passed on.
    When performing a cross merge, no column specifications to merge on are
    allowed.
    
    Parameters
    ----------
    left : DataFrame
    right : DataFrame or named Series
        Object to merge with.
    how : {'left', 'right', 'outer', 'inner', 'cross'}, default 'inner'
        Type of merge to be performed.
    
        * left: use only keys from left frame, similar to 

In [8]:
# Do an inner join!
pd.merge(registrations, logins, how='inner', on='name') # Sweet as!!
# Order for inner joins does not matter!

,reg_id,name,log_id
0,1,Andrew,2
1,2,Bobo,4


In [9]:
# Left and right merge use left or right table as a base where it cannot loose values.
pd.merge(registrations, logins, how='left', on='name')

,reg_id,name,log_id
0,1,Andrew,2.0
1,2,Bobo,4.0
2,3,Claire,NaN
3,4,David,NaN


In [10]:
pd.merge(registrations, logins, how='right', on='name')

,reg_id,name,log_id
0,NaN,Xavier,1
1,1.0,Andrew,2
2,NaN,Yolanda,3
3,2.0,Bobo,4


In [12]:
# Just do left always, more logical
# We can also, however, do an outer condition!
# We can make sure that we grab values from both tables (let's say names in a don't exist in b and vice versa)
pd.merge(registrations, logins, how='outer', on='name')
# Order for outer does not really matter.

,reg_id,name,log_id
0,1.0,Andrew,2.0
1,2.0,Bobo,4.0
2,3.0,Claire,NaN
3,4.0,David,NaN
4,NaN,Xavier,1.0
5,NaN,Yolanda,3.0


In [13]:
# Join on an index instead of a column
registrations = registrations.set_index('name')
registrations

,reg_id
name,
Andrew,1
Bobo,2
Claire,3
David,4


In [16]:
# To join on index.
pd.merge(registrations, logins, left_index=True, right_on='name', how='inner')

,reg_id,log_id,name
1,1,2,Andrew
3,2,4,Bobo


In [17]:
registrations = registrations.reset_index()

In [18]:
registrations.columns = ['reg_name', 'reg_id']

In [19]:
# Merge on different cols between DFs
pd.merge(logins, registrations, left_on='name', right_on='reg_name', how='inner')

,log_id,name,reg_name,reg_id
0,2,Andrew,Andrew,1
1,4,Bobo,Bobo,2


In [20]:
# Tag duplicate columns
registrations.columns = ['name', 'id']

In [21]:
logins

,log_id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


In [22]:
logins.columns = ['id', 'name']

In [23]:
logins

,id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


In [24]:
# Auto label for dupe cols.
pd.merge(registrations, logins, how='inner', on='name') # adds id_x, id_y

,name,id_x,id_y
0,Andrew,1,2
1,Bobo,2,4


In [26]:
pd.merge(registrations, logins, how='inner', on='name', suffixes=('_reg', '_log')) # can change prefixes via a tuple

,name,id_reg,id_log
0,Andrew,1,2
1,Bobo,2,4
